# Overview
- lightGBMのハイパラチューニング

# Const

In [64]:
NB = '009'
N_SPLITS = 5
SHOW_LOG = False
VERBOSE = None
EARLY_STOPPING_ROUNDS = 100


PATH_TRAIN = './../data/official/train.csv'
PATH_TEST = './../data/official/test.csv'
PATH_SAMPLE_SUBMITTION = './../data/official/atmaCup8_sample-submission.csv'
SAVE_DIR = f'../data/output_nb/nb{NB}/'

feat_train_only = ['JP_Sales', 'Global_Sales', 'NA_Sales', 'Other_Sales', 'EU_Sales']
feat_common = ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
           'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
           'Rating']
feat_string = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating']

# Import everything I need :)

In [65]:
import os
import time
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor 

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

# My function

In [66]:
def metric(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** .5

def preprocess_User_Score(df):
    '''
    - tbdをnanにする
    - stringをfloatにする
    '''
    mask = df.User_Score.values == 'tbd'
    df.User_Score[mask] = np.nan
    df.User_Score = df.User_Score.values.astype(float)
    return df

def string_encode(df, cols):
    '''
    - np.nanがあれば、'nan'に置き換える
    - label encodingする
    '''
    df[cols] = df[cols].replace(np.nan, 'nan')
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df

def df_preprocessing(df, string_cols):
    df = preprocess_User_Score(df)
    df = string_encode(df, string_cols)
    return df

In [67]:
def run_fold_lgbm(_X_trn, _y_trn, _X_val, _y_val, _X_te, model_config, show_log=True):
    FEAT_STRING = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating']
    
    # train
    model = LGBMRegressor(random_state=2020, **model_config)
    model.fit(_X_trn, _y_trn.values[:, 0],
              categorical_feature=FEAT_STRING,
              eval_set=[(_X_trn, _y_trn), (_X_val, _y_val)],
              verbose=VERBOSE,
              early_stopping_rounds=EARLY_STOPPING_ROUNDS
             )
    
    # predict
    y_trn_pred = model.predict(_X_trn)
    y_val_pred = model.predict(_X_val)
    _y_test_pred = model.predict(_X_te)
    
    # postprocessiing
    y_trn_pred[y_trn_pred <= 1] = 1
    y_val_pred[y_val_pred <= 1] = 1
    
    if show_log:
        print(show_log)
        print(f'score train: {metric(_y_trn, y_trn_pred):.5f}')
        print(f'score valid: {metric(_y_val, y_val_pred):.5f}')
        print('')
    
    return y_trn_pred, y_val_pred, _y_test_pred

def run(X, y, X_te, splitter, use_col, model_config, show_log=True):
    print(f'use_col: {use_col}\n') if show_log else None

    oof = np.zeros(len(X))
    y_test_pred = np.zeros(len(X_te))
    for fold_i, (idx_trn, idx_val) in enumerate(splitter.split(X)):
        if show_log:
            print(f'::Fold {fold_i+1}/{N_SPLITS} start at {time.ctime()}::')
        X_trn, X_val = X.iloc[idx_trn, :], X.iloc[idx_val, :]
        y_trn, y_val = y.iloc[idx_trn], y.iloc[idx_val]
        X_trn = pd.DataFrame(X_trn, columns=X.columns)
        X_val = pd.DataFrame(X_val, columns=X.columns)

    #     # train
        y_trn_pred, y_val_pred, _y_test_pred = run_fold_lgbm(X_trn, y_trn, X_val, y_val, 
                                                             X_te, model_config, show_log=show_log)

        # result
        oof[idx_val] = y_val_pred
        y_test_pred += _y_test_pred / N_SPLITS


    y_test_pred[y_test_pred <= 1] = 1
    return oof, y_test_pred

# Preparation

set

In [68]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

<br>

load dataset

In [69]:
train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
ss = pd.read_csv(PATH_SAMPLE_SUBMITTION)

<br>

preprocess

In [70]:
train = df_preprocessing(train, feat_string)
test = df_preprocessing(test, feat_string)

In [71]:
mask = test.columns != 'Name'
use_col = test.columns[mask].tolist()
use_col

['Platform',
 'Year_of_Release',
 'Genre',
 'Publisher',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count',
 'Developer',
 'Rating']

In [72]:
X = train[use_col].copy()
y = train[['Global_Sales']].copy()
X_te = test[use_col].copy()

# Create Model

# Tuning

In [73]:
splitter = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2020)

In [95]:
# 最小化したい関数
def objective(trial):
    print('')
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 800, 800),
        'max_depth': trial.suggest_int('max_depth', 5, 300),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.1, 1.0, 0.1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'reg_alpha': trial.suggest_discrete_uniform('reg_alpha', 1, 6, 1),
        'reg_lambda': trial.suggest_discrete_uniform('reg_lambda', 1, 6, 1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    
    oof, y_test_pred = run(X, y, X_te, splitter, use_col, params, show_log=SHOW_LOG)
    return metric(y, oof)

In [98]:
%%time
study = optuna.create_study()
study.optimize(objective, n_trials=6000)
print(study.best_trial)
print(f'\n best_params = {study.best_params}')

[I 2020-12-05 08:03:37,285] Finished trial#0 with value: 1.2028490752247527 with parameters: {'n_estimators': 800, 'max_depth': 94, 'subsample': 0.2, 'colsample_bytree': 0.30000000000000004, 'learning_rate': 0.015469605297026488, 'reg_alpha': 2.0, 'reg_lambda': 2.0, 'min_child_samples': 32}. Best is trial#0 with value: 1.2028490752247527.


[I 2020-12-05 08:03:42,630] Finished trial#1 with value: 1.1206638363621224 with parameters: {'n_estimators': 800, 'max_depth': 66, 'subsample': 0.2, 'colsample_bytree': 0.9, 'learning_rate': 0.010340392363442127, 'reg_alpha': 4.0, 'reg_lambda': 6.0, 'min_child_samples': 59}. Best is trial#1 with value: 1.1206638363621224.


[I 2020-12-05 08:03:46,456] Finished trial#2 with value: 1.2603863247603597 with parameters: {'n_estimators': 800, 'max_depth': 97, 'subsample': 0.5, 'colsample_bytree': 0.30000000000000004, 'learning_rate': 0.019478199334969557, 'reg_alpha': 2.0, 'reg_lambda': 5.0, 'min_child_samples': 92}. Best is trial#1 with value: 1.1206638363621224.


[I 2020-12-05 08:03:50,806] Finished trial#3 with value: 1.14104146342533 with parameters: {'n_estimators': 800, 'max_depth': 267, 'subsample': 0.9, 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.011963251687506775, 'reg_alpha': 3.0, 'reg_lambda': 2.0, 'min_child_samples': 60}. Best is trial#1 with value: 1.1206638363621224.


[I 2020-12-05 08:03:55,304] Finished trial#4 with value: 1.1564364532162223 with parameters: {'n_estimators': 800, 'max_depth': 82, 'subsample': 0.4, 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.003704191611400631, 'reg_alpha': 4.0, 'reg_lambda': 6.0, 'min_child_samples': 80}. Best is trial#1 with value: 1.1206638363621224.


...

[I 2020-12-05 10:27:27,457] Finished trial#1860 with value: 1.0979010234380917 with parameters: {'n_estimators': 800, 'max_depth': 15, 'subsample': 1.0, 'colsample_bytree': 1.0, 'learning_rate': 0.0064859707092929965, 'reg_alpha': 5.0, 'reg_lambda': 5.0, 'min_child_samples': 55}. Best is trial#1013 with value: 1.093538456910296.


[I 2020-12-05 10:27:32,174] Finished trial#1861 with value: 1.0973518295951705 with parameters: {'n_estimators': 800, 'max_depth': 53, 'subsample': 0.4, 'colsample_bytree': 0.9, 'learning_rate': 0.007788506886875487, 'reg_alpha': 1.0, 'reg_lambda': 5.0, 'min_child_samples': 58}. Best is trial#1013 with value: 1.093538456910296.


[I 2020-12-05 10:27:35,741] Finished trial#1862 with value: 1.1476432665942624 with parameters: {'n_estimators': 800, 'max_depth': 39, 'subsample': 0.2, 'colsample_bytree': 0.9, 'learning_rate': 0.0072616270842843335, 'reg_alpha': 5.0, 'reg_lambda': 5.0, 'min_child_samples': 17}. Best is trial#1013 with value: 1.093538456910296.


[I 2020-12-05 10:27:42,827] Finished trial#1863 with value: 1.0983807034546318 with parameters: {'n_estimators': 800, 'max_depth': 49, 'subsample': 0.8, 'colsample_bytree': 0.9, 'learning_rate': 0.0058839480338679015, 'reg_alpha': 5.0, 'reg_lambda': 5.0, 'min_child_samples': 60}. Best is trial#1013 with value: 1.093538456910296.


KeyboardInterrupt: 

<br>

再現するか確かめる 

In [101]:
model_config = {'n_estimators': 800, 'max_depth': 55, 'subsample': 0.5, 'colsample_bytree': 0.9, 'learning_rate': 0.006437110612661787, 'reg_alpha': 5.0, 'reg_lambda': 5.0, 'min_child_samples': 57}

In [102]:
%%time
oof, y_test_pred = run(X, y, X_te, splitter, use_col, model_config, show_log=SHOW_LOG)
print(f'oof score: {metric(y, oof):.5f}')

oof score: 1.09354
CPU times: user 54.1 s, sys: 340 ms, total: 54.4 s
Wall time: 4.55 s
